In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


C:\Anaconda3\envs\carnd-term1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting .\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Epoch  1, Batch   1 - Loss: 61719.2969 Validation Accuracy: 0.140625
Epoch  1, Batch   2 - Loss: 37346.3281 Validation Accuracy: 0.164062
Epoch  1, Batch   3 - Loss: 36445.2266 Validation Accuracy: 0.167969
Epoch  1, Batch   4 - Loss: 33193.1992 Validation Accuracy: 0.164062
Epoch  1, Batch   5 - Loss: 28262.4766 Validation Accuracy: 0.156250
Epoch  1, Batch   6 - Loss: 26862.3633 Validation Accuracy: 0.203125
Epoch  1, Batch   7 - Loss: 23232.1250 Validation Accuracy: 0.222656
Epoch  1, Batch   8 - Loss: 21102.5156 Validation Accuracy: 0.238281
Epoch  1, Batch   9 - Loss: 21413

Epoch  1, Batch 114 - Loss:  4460.0889 Validation Accuracy: 0.703125
Epoch  1, Batch 115 - Loss:  4964.8936 Validation Accuracy: 0.695312
Epoch  1, Batch 116 - Loss:  3437.8921 Validation Accuracy: 0.703125
Epoch  1, Batch 117 - Loss:  3977.3843 Validation Accuracy: 0.707031
Epoch  1, Batch 118 - Loss:  3289.3125 Validation Accuracy: 0.707031
Epoch  1, Batch 119 - Loss:  3620.8267 Validation Accuracy: 0.714844
Epoch  1, Batch 120 - Loss:  3541.4031 Validation Accuracy: 0.707031
Epoch  1, Batch 121 - Loss:  3389.4619 Validation Accuracy: 0.714844
Epoch  1, Batch 122 - Loss:  3696.7703 Validation Accuracy: 0.703125
Epoch  1, Batch 123 - Loss:  4837.0562 Validation Accuracy: 0.710938
Epoch  1, Batch 124 - Loss:  2890.9434 Validation Accuracy: 0.710938
Epoch  1, Batch 125 - Loss:  2793.5596 Validation Accuracy: 0.722656
Epoch  1, Batch 126 - Loss:  3005.9832 Validation Accuracy: 0.707031
Epoch  1, Batch 127 - Loss:  3838.3047 Validation Accuracy: 0.726562
Epoch  1, Batch 128 - Loss:  2870.

KeyboardInterrupt: 